<h1 style="text-align: center;">Simulación y aplicaciones en ciencias sociales y experimentales
</h3>
<h3 style="text-align: center;"> Tema 3. Modelos aleatorios </h3>
<h3 style="text-align: center;"> Modelo de Colas (ejercicios propuestos) </h3>

### Ejercicio 

Se propone un ejercicio teórico-práctico. Consiste en idear un modelo para un sistema real aleatorio markoviano, con planteamiento libre. Se valorará positivamente que el ajuste o estimación de los parámetros o variables aleatorias iniciales a partir de muestreos o toma de datos reales. El objetivo del modelo es ayudar a través de las simulaciones a la toma de decisiones óptimas de gestión/utilización del sistema

Se dan dos ejemplos a modo de sugerencias, pero puede ser cualquier otro proceso markoviano similar a los estudiados. 

- Juego de azar sin memoria, en el que el conjunto de estados es finito (ganar, perder, empatar, etc.) y las probabilidades son fijas. A través del modelo se simularía el estado a largo plazo del jugador, así como la estrategia que optimizara un objetivo determinado.  

- Un comercio que se pide en barra (ferretería, cafetería, etc), en la cual se dispone de una (o varias) personas antendiendo, con un tiempo de atención y servicio medio determinado. Cada servicio supone un ingreso económico y la llegada de clientes se produce cada cierto tiempo y se ponen en cola. Si la cola excede de un número de individuos, el cliente abandona el local. Se podría estudiar el ingreso y pérdida diaria por abandono del cliente, así como la conveniencia o no de contratar a otro empleado si su salario es $w$ por día. 

In [ ]:
import numpy as np
import simpy
import matplotlib.pyplot as plt

# Configuración de parámetros
NUM_CAIXAS = 3  # Número inicial de cajas abiertas
TIEMPO_SIMULACION = 480  # Tiempo de simulación en minutos (8 horas)
LLEGADAS_POR_MINUTO = 1.2  # Tasa promedio de llegada de clientes (Poisson)
TIEMPO_ATENCION_PROMEDIO = 5  # Tiempo promedio de atención por cliente (Exponencial)
LIMITE_COLA = 10  # Máxima cantidad de clientes en cola antes de abandono
COSTO_CAJA = 50  # Costo por caja abierta por hora
INGRESO_POR_CLIENTE = 10  # Ingreso por cliente atendido
PERDIDA_POR_ABANDONO = 5  # Pérdida por cliente que abandona

# Variables globales
clientes_atendidos = 0
clientes_abandonados = 0
total_cola = []

def llegada_clientes(env, supermercado):
    """Genera llegadas de clientes al sistema."""
    global clientes_abandonados
    while True:
        yield env.timeout(np.random.exponential(1 / LLEGADAS_POR_MINUTO))
        if len(supermercado.queue) < LIMITE_COLA:
            env.process(atencion_cliente(env, supermercado))
        else:
            clientes_abandonados += 1

def atencion_cliente(env, supermercado):
    """Simula el tiempo de atención de un cliente."""
    global clientes_atendidos
    with supermercado.request() as request:
        yield request
        yield env.timeout(np.random.exponential(TIEMPO_ATENCION_PROMEDIO))
        clientes_atendidos += 1

def simular_supermercado(num_cajas):
    """Simula el supermercado con el número de cajas especificado."""
    global clientes_atendidos, clientes_abandonados, total_cola
    clientes_atendidos = 0
    clientes_abandonados = 0
    total_cola = []

    env = simpy.Environment()
    supermercado = simpy.Resource(env, capacity=num_cajas)
    env.process(llegada_clientes(env, supermercado))

    while env.now < TIEMPO_SIMULACION:
        total_cola.append(len(supermercado.queue))
        yield env.timeout(1)

    return clientes_atendidos, clientes_abandonados, total_cola

# Simulación para distintos números de cajas
resultados = []
for num_cajas in range(1, 6):  # Simular con 1 a 5 cajas abiertas
    clientes_atendidos, clientes_abandonados, total_cola = simular_supermercado(num_cajas)
    ingreso_total = clientes_atendidos * INGRESO_POR_CLIENTE
    costo_total = num_cajas * COSTO_CAJA * (TIEMPO_SIMULACION / 60)
    perdida_total = clientes_abandonados * PERDIDA_POR_ABANDONO
    beneficio_neto = ingreso_total - costo_total - perdida_total
    resultados.append((num_cajas, ingreso_total, costo_total, perdida_total, beneficio_neto))

# Presentar los resultados
import pandas as pd
df_resultados = pd.DataFrame(resultados, columns=["Cajas Abiertas", "Ingreso Total", "Costo Total", "Pérdida Total", "Beneficio Neto"])
import ace_tools as tools; tools.display_dataframe_to_user(name="Resultados de la Simulación del Supermercado", dataframe=df_resultados)

# Graficar las colas promedio para cada número de cajas
plt.figure(figsize=(10, 6))
for i, (num_cajas, _, _, _, _) in enumerate(resultados):
    if i == 0:
        clientes_atendidos, clientes_abandonados, total_cola = simular_supermercado(num_cajas)
    plt.plot(total_cola, label=f"{num_cajas} Cajas Abiertas")

plt.xlabel("Tiempo (minutos)")
plt.ylabel("Tamaño de la Cola")
plt.title("Tamaño de la Cola en Función del Tiempo")
plt.legend()
plt.show()
